<H1>Output parsers</H1>

언어 모델들이 출력하는 text, 즉 response를 구조화하는 기능이다.
Output parser가 반드시 구현해야 하는 2개의 main method가 있다.
- "Get format instructions": 언어 모델의 출력 형식을 지정하는 방법에 대한 지침이 포함된 문자열을 반환하는 메소드
- "Parse": 언어 모델의 응답(text)을 가져와서 일부 구조로 구문 분석하는 방법

그리고 선택적으로 구현해야 하는 1개의 메소드가 있다.
- "Parse with prompt": 언어 모델의 응답(text)와 그 응답을 만들어낸 prompt를 받아서 일부 구조로 구문 분석하는 방법. prompt는 주로 OutputParser가 어떤 방식으로든 출력을 재시도하거나 수정하려는 경우에 제공되고, 그렇게 하려면 prompt의 정보가 필요하다.

종류가 여러가지가 있어서 list 형태, 날짜 형태, enum 형태, JSON 형태, 그리고 답변 형식이 잘못됐을 땐 OutputFixingParser, 어디가 잘못됐는지도 모르게 잘못된 답변은 RetryOutputParser 등이 있다.

In [1]:
from dotenv import load_dotenv
load_dotenv('config.env')

True

In [2]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

model_name = 'text-davinci-003'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [3]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

In [4]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

output = model(_input.to_string())

parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')